# 트랜스포머 (Transformer)

* 참고: https://wikidocs.net/31379

* attention mechanism은 seq2seq의 입력 시퀀스 정보 손실을 보정해주기 위해 사용됨
* attention mechanism을 보정 목적이 아닌, 인코더와 디코더로 구성한 모델이 바로 트랜스포머
* 트랜스포머는 RNN을 사용하지 않고 인코더와 디코더를 설계하였으며, 성능도 RNN보다 우수함



## 포지셔널 인코딩

* 기존의 RNN은 단어의 위치를 따라 순차적으로 입력받아 단어의 위치정보를 활용할 수 있었음
* 트랜스포머의 경우, RNN을 활용하지 않았기 때문에 단어의 위치정보를 다른 방식으로 줄 필요가 있음
* 이를 위해 **각 단어의 임베딩 벡터에 위치 정보들을 더하게 되는데** 이를 포지셔널 인코딩이라 함
* 보통 포지셔널 인코딩은 sin, cos을 이용하여 계산

In [1]:
def positional_encoding(dim, sentence_length):
  encoded_vec = np.array([pos / np.power(10000, 2 * i / dim) for pos in range(sentence_length) for i in range(dim)])
  encoded_vec[::2] = np.sin(encoded_vec[::2])
  encoded_vec[1::2] = np.cos(encoded_vec[1::2])
  
  return tf.constant(encoded_vec.reshape([sentence_length, dim]), dtype=tf.float32)

## 레이어 정규화

*  레이어 정규화에서는 텐서의 마지막 차원에 대해 평균과 분산을 구하고, 이 값을 통해 값을 정규화함
*  해당 정규화를 각 층의 연결에 편리하게 적용하기 위해 함수화한 `sublayer_connection()`을 선언

In [2]:
def layer_norm(inputs, eps=1e-6):
  feature_shape = inputs.get_shape()[-1:]
  mean = tf.keras.backend.mean(inputs, [-1], keepdims=True)
  std = tf.keras.backend.std(inputs, [-1], keepdims=True)
  beta = tf.Variable(tf.zeros(feature_shape), trainable=False)
  gamma = tf.Variable(tf.ones(feature_shape), trainable=False)
  return gamma * (inputs - mean) / (std + eps) + beta

In [3]:
def sublayer_connection(inputs, sublayer, dropout=0.2):
  outputs = layer_norm(inputs + tf.keras.layers.Dropout(dropout)(sublayer))
  return outputs

## 어텐션



*   트랜스포머 모델의 핵심이 되는 부분
*   트랜스포머에서는 multi-head attention과 self attention이라는 개념을 사용
  1.   multi-head attention
      * 디코더가 가지는 차원을 나누어 병렬로 어텐션을 진행
      *  마지막엔 병렬로 각 진행해 얻은 어텐션 헤드를 모두 연결
      * 이로 인해 다양한 시각에서 정보를 수집할 수 있는 효과를 얻음
  2.   self attention
      *   일반적인 어텐션의 경우, 특정 시점의 디코더 은닉상태와 모든 시점의 인코더 은닉상태를 활용
      *   이는 입력 문장과 다른 문장에 존재하는 단어간의 어텐션을 의미함
      *   반면 self attention은 은닉 상태를 동일하게 하여 어텐션을 진행
      *   이는 입력 문장 내 단어간의 어텐션을 의미함




*   트랜스포머 제안 논문에서는 scaled-dot product attention을 활용해 모델을 작성함



### scaled-dot product attention 구현

* scaled-dot product attention은 앞서 학습한 dot product attention과 거의 유사함
* 단 attention을 진행할 때 어텐션 스코어를 계산할 때 내적 값을 정규화
* 트랜스포머에서는 정규화할 때 K 벡터(=디코더 셀의 은닉 상태)의 차원을 루트를 취한 값을 사용

In [4]:
def scaled_dot_product_attention(query, key, value, masked=False):
  key_dim_size = float(key.get_shape().as_list()[-1])
  key = tf.transpose(key, perm=[0, 2, 1])

  outputs = tf.matmul(query, key) / tf.sqrt(key_dim_size)

  if masked:
    diag_vals = tf.ones_like(outputs[0, :, :])
    tril = tf.linalg.LinearOperatorLowerTriangular(diag_vals).to_dense()
    masks = tf.tile(tf.expand_dims(tril, 0), [tf.shape(outputs)[0], 1, 1])
    paddings = tf.ones_like(masks) * (-2**30)
    outputs = tf.where(tf.equal(masks, 0), paddings, outputs)

  attention_map = tf.nn.softmax(outputs)
  return tf.matmul(attention_map, value)

### multi-head attention 구현

* multi-head attention의 구현 과정
  1. query, key, value에 해당하는 값을 받고, 해당 값에 해당하는 행렬 생성
  2. 생성된 행렬들을 heads에 해당하는 수만큼 분리
  3. 분리한 행렬들에 대해 각각 어텐션을 수행
  4. 각 어텐션 결과들을 연결해 최종 어텐션 결과 생성





In [5]:
def multi_head_attention(query, key, value, num_units, heads, masked=False):
  query = tf.keras.layers.Dense(num_units, activation=tf.nn.relu)(query)
  key = tf.keras.layers.Dense(num_units, activation=tf.nn.relu)(key)
  value = tf.keras.layers.Dense(num_units, activation=tf.nn.relu)(value)
  
  query = tf.concat(tf.split(query, heads, axis=-1), axis=0)
  key = tf.concat(tf.split(key, heads, axis=-1), axis=0)
  value = tf.concat(tf.split(value, heads, axis=-1), axis=0)
  
  attention_map = scaled_dot_product_attention(query, key, value, masked)
  attn_outputs = tf.concat(tf.split(attention_map, heads, axis=0), axis=-1)
  attn_outputs = tf.keras.layers.Dense(num_units, activation=tf.nn.relu)(attn_outputs)

  return attn_outputs

## 포지션-와이즈 피드 포워드 신경망



*   multi-head attention의 결과인 행렬을 입력받아 연산
*   일반적인 완전 연결 신경망(Dense layer)를 사용
*   position-wise FFNN은 인코더와 디코더에 모두 존재



In [6]:
def feed_forward(inputs, num_units):
  feature_shape = inputs.get_shape()[-1]
  inner_layer = tf.keras.layers.Dense(num_units, activation=tf.nn.relu)(inputs)
  outputs = tf.keras.layers.Dense(feature_shape)(inner_layer)
  return outputs

## 인코더


* 인코더는 하나의 어텐션을 사용
  + encoder self-attention (multi-head self-attention과 동일)

In [7]:
def encoder_module(inputs, model_dim, ffn_dim, heads):
  self_attn = sublayer_connection(inputs, multi_head_attention(inputs, inputs, inputs, model_dim, heads))
  outputs = sublayer_connection(self_attn, feed_forward(self_attn, ffn_dim))
  return outputs

def encoder(inputs, model_dim, ffn_dim, heads, num_layers):
  outputs = inputs
  for i in range(num_layers):
    outputs = encoder_module(outputs, model_dim, ffn_dim, heads)
  
  return outputs

## 디코더

* 디코더는 다음과 같은 구성의 반복으로 이루어짐
  1. masked decoder self-attention
  2. encoder-decoder attention
  3. position-wise FFNN

* 디코더에서는 2종류의 어텐션을 사용
  1.   masked decoder self-attention
    *   디코더에서는 인코더와는 달리 순차적으로 결과를 만들어 내야하기 때문에 다른 어텐션 방법을 사용함
    *   디코더 예측 시점 이후의 위치에 attention을 할 수 없도록 masking 처리
    *   결국 예측 시점에서 예측은 미리 알고 있는 위치까지만의 결과에 의존
  2.   encoder-decoder attention
    *   앞서 설명한 multi-head attention과 동일



In [8]:
def decoder_module(inputs, encoder_outputs, model_dim, ffn_dim, heads):
  masked_self_attn = sublayer_connection(inputs, 
                                         multi_head_attention(inputs, inputs, inputs, 
                                                              model_dim, heads, masked=True))
  self_attn = sublayer_connection(masked_self_attn, 
                                  multi_head_attention(masked_self_attn, 
                                                       encoder_outputs, encoder_outputs, model_dim, heads))
  
  outputs = sublayer_connection(self_attn, feed_forward(self_attn, ffn_dim))
  return outputs

def decoder(inputs, encoder_outputs, model_dim, ffn_dim, heads, num_layers):
  outputs = inputs
  for i in range(num_layers):
    outputs = decoder_module(outputs, encoder_outputs, model_dim, ffn_dim, heads)
  
  return outputs

## 트랜스포머를 활용한 챗봇

### konlpy 라이브러리

*    한글을 처리하기 위해 konlpy 라이브러리 설치

In [9]:
!pip install konlpy

### 데이터 준비

* 처리에 필요한 각종 변수 선언
* filters에 해당되는 문자를 걸러주는 정규 표현식 컴파일



In [10]:
import re
import tensorflow as tf

filters = "([~.,!?\"':;)(])"
PAD = '<PADDING>'
STD = '<START>'
END = '<END>'
UNK = '<UNKOWN>'

PAD_INDEX = 0
STD_INDEX = 1
END_INDEX = 2
UNK_INDEX = 3

MARKER = [PAD, STD, END, UNK]
CHANGE_FILTER = re.compile(filters)

* 주소에서 데이터를 가져오는 `load_data()` 함수 선언



In [11]:
from sklearn.model_selection import train_test_split

def load_data(data_path):
  data_df = pd.read_csv(data_path, header=0)
  question, answer = list(data_df['Q']), list(data_df['A'])
  train_input, eval_input, train_label, eval_label = train_test_split(question, answer, test_size=0.33, random_state=111)
  
  return train_input, train_label, eval_input, eval_label

* 처리에 필요한 단어 사전을 생성하는 `load_vocab()` 함수 선언

In [12]:
def load_vocabulary(data_path):
  data_df = pd.read_csv(data_path, encoding='utf-8')
  question, answer = list(data_df['Q']), list(data_df['A'])
  if tokenize_as_morph:
    question = prepro_like_morphlized(question)
    answer = prepro_like_morphlized(answer)
  
  data = []
  data.extend(question)
  data.extend(answer)
  words = data_tokenizer(data)
  words = list(set(words))
  words[:0] = MARKER

  char2idx = {char:idx for idx, char in enumerate(words)}
  idx2char = {idx:char for idx, char in enumerate(words)}
  return char2idx, idx2char, len(char2idx)

* 문자열 데이터를 학습에 사용될 수 있도록 변현하는 `prepro_like_morphlized()` 함수 선언



In [13]:
from konlpy.tag import Okt

def prepro_like_morphlized(data):
  morph_analyzer = Okt()
  result_data = list()

  for seq in data:
    morphlized_seq = ' '.join(morph_analyzer.morphs(seq.replace(' ', '')))
    result_data.append(morphlized_seq)
  
  return result_data

* 단어 사전을 만들기 위해 단어들을 분리하는 `data_tokenizer()` 함수 선언

In [14]:
def data_tokenizer(data):
  words = []
  for sentence in data:
    sentence = re.sub(CHANGE_FILTER, '', sentence)
    for word in sentence.split():
      words.append(word)
  
  return [word for word in words if word]

* encoder의 입력을 구성하기 위한 함수 `enc_processing()` 선언



In [15]:
def enc_processing(value, dictionary):
  sequences_input_index = []
  sequences_length = []

  if tokenize_as_morph:
    value = prepro_like_morphlized(value)
  
  for sequence in value:
    sequence = re.sub(CHANGE_FILTER, '', sequence)
    sequence_index = []
    for word in sequence.split():
      if dictionary.get(word) is not None:
        sequence_index.extend([dictionary[word]])
      else:
        sequence_index.extend([dictionary[UNK]]) 
    
    if len(sequence_index) > max_len:
      sequence_index = sequence_index[:max_len]
    sequences_length.append(len(sequence_index))
    sequence_index += (max_len - len(sequence_index)) * [dictionary[PAD]]
    sequences_input_index.append(sequence_index)
  return np.asarray(sequences_input_index), sequences_length

* decoder의 입력을 구성하기 위한 함수 `dec_output_processing()` 선언

In [16]:
def dec_output_processing(value, dictionary):
  sequences_output_index = []
  sequences_length = []

  if tokenize_as_morph:
    value = prepro_like_morphlized(value)
  
  for sequence in value:
    sequence = re.sub(CHANGE_FILTER, '', sequence)
    sequence_index = []
    sequence_index = [dictionary[STD]] + [dictionary[word] for word in sequence.split()]
    
    if len(sequence_index) > max_len:
      sequence_index = sequence_index[:max_len]
    sequences_length.append(len(sequence_index))
    sequence_index += (max_len - len(sequence_index)) * [dictionary[PAD]]
    sequences_output_index.append(sequence_index)
  return np.asarray(sequences_output_index), sequences_length

* decoder의 출력을 구성하기 위한 함수 `dec_target_processing()` 선언

In [17]:
def dec_target_processing(value, dictionary):
  sequences_target_index = []

  if tokenize_as_morph:
    value = prepro_like_morphlized(value)
  
  for sequence in value:
    sequence = re.sub(CHANGE_FILTER, '', sequence)
    sequence_index = []
    sequence_index = [dictionary[word] for word in sequence.split()]
    
    if len(sequence_index) >= max_len:
      sequence_index = sequence_index[:max_len - 1] + [dictionary[END]]
    else:
      sequence_index += [dictionary[END]]

    sequence_index += (max_len - len(sequence_index)) * [dictionary[PAD]]
    sequences_target_index.append(sequence_index)
  return np.asarray(sequences_target_index)

* 모델에 데이터를 효율적으로 투입하도록 `train_input_fn()`, `eval_input_fn()` 함수 선언
* `rearrange()`는 dataset 객체가 데이터를 어떻게 변형시킬지 정의해둔 함수
* dataset.map은 rearrange 함수를 기반으로 데이터를 변형



In [18]:
def train_input_fn(train_input_enc, train_output_enc, train_target_dec, batch_size):
  dataset = tf.compat.v1.data.Dataset.from_tensor_slices((train_input_enc, train_output_enc, train_target_dec))
  dataset = dataset.shuffle(buffer_size = len(train_input_enc))
  dataset = dataset.batch(batch_size)
  dataset = dataset.map(rearange)
  dataset = dataset.repeat()
  iterator = dataset.make_one_shot_iterator()
  return iterator.get_next()

def eval_input_fn(eval_input_enc, eval_output_enc, eval_target_dec, batch_size):
  dataset = tf.compat.v1.data.Dataset.from_tensor_slices((eval_input_enc, eval_output_enc, eval_target_dec))
  dataset = dataset.shuffle(buffer_size = len(eval_input_enc))
  dataset = dataset.batch(batch_size)
  dataset = dataset.map(rearange)
  dataset = dataset.repeat(1)
  iterator = dataset.make_one_shot_iterator()
  return iterator.get_next()

def rearange(input, output, target):
  features = {'input':input, 'output':output}
  return features, target

* 모델의 예측은 배열로 생성되기 때문에 이를 확인하기 위해선 문자열로 변환이 필요
* 예측을 문자열로 변환해주는 `pred2string()` 함수 선언


In [19]:
def pred2string(value, dictionary):
  sentence_string = []
  is_finished = False

  for v in value:
    sentence_string = [dictionary[index] for index in v['indexs']]

  answer = ''
  for word in sentence_string:
    if word == END:
      is_finished = True
      break
    
    if word != PAD and word != END:
      answer += word
      answer += ' '

  return answer, is_finished

* 챗봇 데이터 URL: https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData%20.csv
* 데이터 주소에서 데이터를 읽어들여 단어 사전과 사용 데이터 구성

In [20]:
import pandas as pd

tokenize_as_morph = True

data_path = 'https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData%20.csv'

char2idx, idx2char, len_vocab = load_vocabulary(data_path)
train_input, train_label, eval_input, eval_label = load_data(data_path)

### 모델 구성

* 앞서 작성한 트랜스포머 모델을 결합해 학습에 사용할 모델을 구성함

In [21]:
def model(features, labels, mode, params):
  TRAIN = mode == tf.estimator.ModeKeys.TRAIN
  EVAL = mode == tf.estimator.ModeKeys.EVAL
  PREDICT = mode == tf.estimator.ModeKeys.PREDICT
  
  position_encode = positional_encoding(params['embedding_size'], params['max_len'])
  if params['xavier_initializer']:
    embedding_initializer = 'glorot_normal'
  else:
    embedding_initializer = 'uniform'
  
  embedding = tf.keras.layers.Embedding(params['len_vocab'], 
                                        params['embedding_size'],
                                        embeddings_initializer=embedding_initializer)
  
  x_embedded_matrix = embedding(features['input']) + position_encode
  y_embedded_matrix = embedding(features['output']) + position_encode

  encoder_outputs = encoder(x_embedded_matrix, params['model_hidden_size'], params['ffn_hidden_size'], 
                            params['attention_head_size'], params['layer_size'])
  decoder_outputs = decoder(y_embedded_matrix, encoder_outputs, params['model_hidden_size'], 
                            params['ffn_hidden_size'], params['attention_head_size'], params['layer_size'])
  
  logits = tf.keras.layers.Dense(params['len_vocab'])(decoder_outputs)
  predict = tf.argmax(logits, 2)

  if PREDICT:
    predictions = {'indexs':predict, 
                   'logits':logits}
    return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  labels_ = tf.one_hot(labels, params['len_vocab'])
  loss = tf.reduce_mean(tf.compat.v1.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels_))
  accuracy = tf.compat.v1.metrics.accuracy(labels=labels, predictions=predict)

  metrics = {'accuracy':accuracy}
  tf.summary.scalar('accuracy', accuracy[1])

  if EVAL:
    return tf.estimator.EstimatorSpec(mode, loss=loss, eval_metric_ops=metrics)
  assert TRAIN

  optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=params['learning_rate'])
  train_op = optimizer.minimize(loss, global_step=tf.compat.v1.train.get_global_step())
  return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

### 모델 학습

*   필요한 각종 인자들을 설정
*   인자에 따라 학습 결과가 달라질 수 있기 때문에 세심한 조정이 필요


In [22]:
max_len = 25
epoch = 5000
batch_size = 256
embedding_size = 100
model_hidden_size = 100
ffn_hidden_size = 100
attention_head_size = 100
lr = 0.001
layer_size = 3
xavier_initializer = True

*   앞서 선언한 processing 함수로 데이터를 모델에 투입할 수 있도록 가공
*   평가 데이터에도 동일하게 가공

In [23]:
import numpy as np

train_input_enc, train_input_enc_length = enc_processing(train_input, char2idx)
train_output_dec, train_output_dec_length = dec_output_processing(train_label, char2idx)
train_target_dec = dec_target_processing(train_label, char2idx)

eval_input_enc, eval_input_enc_length = enc_processing(eval_input, char2idx)
eval_output_dec, eval_output_dec_length = dec_output_processing(eval_label, char2idx)
eval_target_dec = dec_target_processing(eval_label, char2idx)

* 앞서 선언한 함수를 통해 모델을 선언하고 학습
* `tf.estimator`를 사용해 간편하게 학습 모듈 구성


In [24]:
transformer = tf.estimator.Estimator(
    model_fn = model,
    params = {'embedding_size':embedding_size, 
              'model_hidden_size':model_hidden_size,
              'ffn_hidden_size':ffn_hidden_size,
              'attention_head_size':attention_head_size,
              'learning_rate':lr,
              'len_vocab':len_vocab,
              'layer_size':layer_size,
              'max_len':max_len,
              'xavier_initializer':xavier_initializer}
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp7e9vc1ek', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


* 학습한 모델을 사용해 챗봇을 사용
* 예측 결과를 문자열로 변환할 때는 앞서 선언한 `pred2string()` 함수를 이용
* 입력에 대한 응답이 생성되는 것을 확인할 수 있음


In [25]:
transformer.train(input_fn=lambda: train_input_fn(train_input_enc, train_output_dec, train_target_dec, batch_size), steps = epoch)
eval_result = transformer.evaluate(input_fn=lambda: eval_input_fn(eval_input_enc, eval_output_dec, eval_target_dec, batch_size))
print('accuracy: {accuracy: 0.3f}'.format(**eval_result))

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
This is a deprecated API that should only be used in TF 1 graph mode and legacy TF 2 graph mode available through `tf.compat.v1`. In all other situations -- namely, eager mode and inside `tf.function` -- you can consume dataset elements using `for elem in dataset: ...` or by explicitly creating iterator via `iterator = iter(dataset)` and fetching its elements via `values = next(iterator)`. Furthermore, this API is not available in TF 2. During the transition from TF 1 to TF 2 you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)` to create a TF 1 graph mode style iterator for a dataset created through TF 2 APIs. Note that this should be a transient state of your code base as there are in general no guarantees about the interoperability of TF 1 and TF 2 code.
INFO:tensorflow:Calling model_fn.
Instructio

### 예측

* 학습한 모델을 사용해 챗봇을 사용
* 예측 결과를 문자열로 변환할 때는 앞서 선언한 `pred2string()` 함수를 이용
* 입력에 대한 응답이 생성되는 것을 확인할 수 있음


In [26]:
def chatbot(sentence):
  pred_input_enc, pred_input_enc_length = enc_processing([sentence], char2idx)
  pred_output_dec, pred_output_dec_length = dec_output_processing([''], char2idx)
  pred_target_dec = dec_target_processing([''], char2idx)

  for i in range(max_len):
    if i > 0:
      pred_output_dec, pred_output_dec_length = dec_output_processing([answer], char2idx)
      pred_target_dec = dec_target_processing([answer], char2idx)
    
    predictions = transformer.predict(input_fn=lambda: eval_input_fn(pred_input_enc, pred_output_dec, pred_target_dec, 1))

    answer, finished = pred2string(predictions, idx2char)

    if finished:
      break

  return answer

In [27]:
chatbot('안녕?')

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp7e9vc1ek/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp7e9vc1ek/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


'안녕하세요 '

In [28]:
chatbot('너 누구냐?')

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp7e9vc1ek/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp7e9vc1ek/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp7e9vc1ek/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp7e9vc1ek/model.ckpt-5000
INFO:tensor

'좋은 방법 이란 없겠지요 '

In [29]:
chatbot('뭐 먹었어?')

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp7e9vc1ek/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp7e9vc1ek/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp7e9vc1ek/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp7e9vc1ek/model.ckpt-5000
INFO:tensor

'저 는 배터리 가 밥 이 예요 '

In [30]:
chatbot('놀고 싶다.')

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp7e9vc1ek/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp7e9vc1ek/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp7e9vc1ek/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp7e9vc1ek/model.ckpt-5000
INFO:tensor

'어느 것 들 이 힘들겠네요 '

In [31]:
chatbot('이제 그만 잘래')

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp7e9vc1ek/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp7e9vc1ek/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp7e9vc1ek/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp7e9vc1ek/model.ckpt-5000
INFO:tensor

'그런 날 이 있더라고요 '